# Part 1

In [1]:
from dataclasses import dataclass
import re

@dataclass
class Group:
    id_: int
    num_units: int
    hit_points: int
    attack_damage: int
    attack_type: str
    initiative: int
    weaknesses: set
    immunities: set
    army: str
    
    @property
    def effective_power(self):
        return self.num_units * self.attack_damage
    
    def __hash__(self):
        ''' Make sure that groups can be put into dicts and sets. '''
        return hash((self.id_, self.army))

    def __repr__(self):
        return '{} Group {}<Units={} HP={} Attack={}/{} Init={} Weak={} Immune={}>'.format(
            self.army, self.id_, self.num_units, self.hit_points, 
            self.attack_damage, self.attack_type, self.initiative,
            ','.join(self.weaknesses) if self.weaknesses else 'NA',
            ','.join(self.immunities) if self.immunities else 'NA')
    
group_re = re.compile(r'^(\d+) units each with (\d+) hit points (\(((weak|immune) '
                      r'to ([a-z, ]+))?;? ?((weak|immune) to ([a-z, ]+))?\))? ?'
                      r'with an attack that does (\d+) (\w+) damage at initiative (\d+)$')

def parse_group(line, id_, army):
    ''' Parse line of text and return a Group. '''
    match = group_re.match(line)
    if not match:
        raise Exception('Cannot parse group: {}'.format(line))
    num_units = int(match.group(1))
    hit_points = int(match.group(2))
    weaknesses, immunities = set(), set()
    if match.group(5) == 'weak':
        weaknesses = {w.strip() for w in match.group(6).split(',')} if match.group(4) else set()
    elif match.group(5) == 'immune':
        immunities = {i.strip() for i in match.group(6).split(',')} if match.group(4) else set()
    if match.group(8) == 'weak':
        weaknesses = {w.strip() for w in match.group(9).split(',')} if match.group(6) else set()
    elif match.group(8) == 'immune':
        immunities = {i.strip() for i in match.group(9).split(',')} if match.group(6) else set()
    attack_damage = int(match.group(10))
    attack_type = match.group(11)
    initiative = int(match.group(12))
    return Group(id_, num_units, hit_points, attack_damage, attack_type, initiative,
                 weaknesses, immunities, army)

In [2]:
g = parse_group('18 units each with 729 hit points (weak to fire; '
            'immune to cold, slashing) with an attack that does '
            '8 radiation damage at initiative 10', 1, 'Infection')
g

Infection Group 1<Units=18 HP=729 Attack=8/radiation Init=10 Weak=fire Immune=cold,slashing>

In [3]:
g.effective_power

144

In [4]:
print(parse_group('17 units each with 5390 hit points (weak to radiation, bludgeoning) '
                  'with an attack that does 4507 fire damage at initiative 2', 1, 'Infection'))
print(parse_group('989 units each with 1274 hit points (immune to fire) '
                  'with an attack that does 25 slashing damage at initiative 3', 2, 'Infection'))
print(parse_group('989 units each with 1274 hit points (immune to fire; '
                  'weak to bludgeoning, slashing) with an attack that does '
                  '25 slashing damage at initiative 3', 3, 'Infection'))
print(parse_group('550 units each with 5068 hit points with an attack that '
                  'does 87 radiation damage at initiative 19', 4, 'Infection'))

Infection Group 1<Units=17 HP=5390 Attack=4507/fire Init=2 Weak=bludgeoning,radiation Immune=NA>
Infection Group 2<Units=989 HP=1274 Attack=25/slashing Init=3 Weak=NA Immune=fire>
Infection Group 3<Units=989 HP=1274 Attack=25/slashing Init=3 Weak=slashing,bludgeoning Immune=fire>
Infection Group 4<Units=550 HP=5068 Attack=87/radiation Init=19 Weak=NA Immune=NA>


In [5]:
import itertools

def parse_text(text):
    ''' Parse text and return two armies. '''
    immune_system = list()
    infection = list()
    current = None
    for line in text.strip().split('\n'):
        if line == 'Immune System:':
            current = immune_system
            id_counter = itertools.count()
            next(id_counter)
            army = 'Immune System'
        elif line == 'Infection:':
            current = infection
            id_counter = itertools.count()
            next(id_counter)
            army = 'Infection'
        elif line.strip() == '':
            pass
        else:
            current.append(parse_group(line.strip(), next(id_counter), army))
    return immune_system, infection

In [6]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)

In [7]:
# Can we parse the entire problem input?
with open('input.txt') as input_:
    text = input_.read()
immune_system, infection = parse_text(text)
print('\n'.join(map(str,immune_system)))
print()
print('\n'.join(map(str,infection)))

Immune System Group 1<Units=9936 HP=1739 Attack=1/slashing Init=11 Weak=slashing,fire Immune=NA>
Immune System Group 2<Units=2990 HP=9609 Attack=31/cold Init=1 Weak=radiation Immune=cold,fire>
Immune System Group 3<Units=2637 HP=9485 Attack=26/radiation Init=13 Weak=bludgeoning Immune=cold,slashing>
Immune System Group 4<Units=1793 HP=2680 Attack=13/bludgeoning Init=10 Weak=bludgeoning Immune=cold>
Immune System Group 5<Units=8222 HP=6619 Attack=6/bludgeoning Init=12 Weak=NA Immune=slashing,fire>
Immune System Group 6<Units=550 HP=5068 Attack=87/radiation Init=19 Weak=NA Immune=NA>
Immune System Group 7<Units=950 HP=8681 Attack=73/slashing Init=17 Weak=radiation Immune=NA>
Immune System Group 8<Units=28 HP=9835 Attack=2979/bludgeoning Init=3 Weak=NA Immune=NA>
Immune System Group 9<Units=3799 HP=2933 Attack=7/slashing Init=16 Weak=NA Immune=NA>
Immune System Group 10<Units=35 HP=8999 Attack=2505/cold Init=6 Weak=bludgeoning Immune=radiation>

Infection Group 1<Units=1639 HP=28720 Attac

In [8]:
def select_target(group, targets, debug=False):
    ''' Select a target for group or None if no target is available. '''
    target_order = [(get_damage(group, target), target.effective_power, target.initiative, target) 
                    for target in targets]
    target_order.sort(reverse=True)
    if debug:
        for damage, _, _, target in target_order:
            print('{} group {} would deal defending group {} {} damage'.format(
                group.army, group.id_, target.id_, damage))
    try:
        damage, _, _, target = target_order[0]
    except IndexError:
        # No targets left
        return None
    if damage > 0:
        targets.remove(target)
        return target
    else:
        return None

def get_damage(group, target):
    ''' Return damage amount inflicted when group attacks target. '''
    if group.attack_type in target.immunities:
        return 0
    elif group.attack_type in target.weaknesses:
        return group.effective_power * 2
    else:
        return group.effective_power
    
class WarIsDeadlocked(Exception):
    pass

def fight_round(immune_system, infection, debug=False):
    ''' Conduct one round of the fight using a list of immune system groups and
    a list of infection groups. Note: this modifies the input lists! '''
    if debug:
        print('Immune system:')
        for group in immune_system:
            print('Group {} contains {} units'.format(group.id_, group.num_units))
        print('Infection:')
        for group in infection:
            print('Group {} contains {} units'.format(group.id_, group.num_units))
        print()

    # Target selection phase
    groups = immune_system + infection
    target_order = [(g.effective_power, g.initiative, g) for g in groups]
    target_order.sort(reverse=True)
    targets = dict()
    immune_system_remaining = list(immune_system)
    infection_remaining = list(infection)

    for _, _, group in target_order:
        if group in immune_system:
            target = select_target(group, infection_remaining, debug)
        elif group in infection:
            target = select_target(group, immune_system_remaining, debug)
        else:
            raise Exception('Should not happen')
        targets[group] = target
        
    if debug:
        print()

    # After working on part 2, I realized that there are scenarios where the battle
    # never finishes because the armies get deadlocked: the only units that remain
    # are all immune to the attack types of the enemies. If we detect that condition,
    # then we should bail early.
    target_count = sum(0 if target is None else 1 for target in targets.values())
    if target_count == 0:
        raise WarIsDeadlocked('No targets can be found')

    # Attacking phase
    attack_order = [(g.initiative, g) for g in groups]
    attack_order.sort(reverse=True)
    total_units_destroyed = 0
    
    for _, group in attack_order:
        if group.num_units == 0:
            continue
        target = targets[group]
        if target is None:
            continue
        damage = get_damage(group, target)
        # Figure out how many units to remove.
        units_destroyed = min(damage // target.hit_points, target.num_units)
        total_units_destroyed += units_destroyed
        target.num_units -= units_destroyed
        if debug:
            group_name = 'Immune System' if group in immune_system else 'Infection'
            print('{} group {} attacks defending group {}, killing {} units'.format(
                group_name, group.id_, target.id_, units_destroyed))

    if debug:
        print()
    # Another way the war can get deadlocked: if none of the remaining units can do
    # enough damage to destroy any of the enemy units. We detect that condition and
    # raise an exception.
    if total_units_destroyed == 0:
        raise WarIsDeadlocked('No units can be destroyed')
    
    # Remove defeated units:
    for idx in range(len(immune_system)-1, -1, -1):
        if immune_system[idx].num_units == 0:
            del immune_system[idx]
    for idx in range(len(infection)-1, -1, -1):
        if infection[idx].num_units == 0:
            del infection[idx]

In [9]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
# Note that the target selection round order of statements differs from the
# sample problem, but the values are correct.
fight_round(immune_system, infection, debug=True)

Immune system:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Infection group 1 would deal defending group 1 185832 damage
Infection group 1 would deal defending group 2 185832 damage
Immune System group 1 would deal defending group 2 153238 damage
Immune System group 1 would deal defending group 1 76619 damage
Infection group 2 would deal defending group 2 107640 damage
Immune System group 2 would deal defending group 1 24725 damage

Infection group 2 attacks defending group 2, killing 84 units
Immune System group 2 attacks defending group 1, killing 4 units
Immune System group 1 attacks defending group 2, killing 51 units
Infection group 1 attacks defending group 1, killing 17 units



In [10]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 905 units
Infection:
Group 1 contains 797 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 184904 damage
Immune System group 2 would deal defending group 1 22625 damage
Immune System group 2 would deal defending group 2 22625 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 144 units



In [11]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 761 units
Infection:
Group 1 contains 793 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183976 damage
Immune System group 2 would deal defending group 1 19025 damage
Immune System group 2 would deal defending group 2 19025 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 143 units



In [12]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 618 units
Infection:
Group 1 contains 789 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183048 damage
Immune System group 2 would deal defending group 1 15450 damage
Immune System group 2 would deal defending group 2 15450 damage

Immune System group 2 attacks defending group 1, killing 3 units
Infection group 1 attacks defending group 2, killing 143 units



In [13]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 475 units
Infection:
Group 1 contains 786 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 182352 damage
Immune System group 2 would deal defending group 1 11875 damage
Immune System group 2 would deal defending group 2 11875 damage

Immune System group 2 attacks defending group 1, killing 2 units
Infection group 1 attacks defending group 2, killing 142 units



In [14]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 333 units
Infection:
Group 1 contains 784 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181888 damage
Immune System group 2 would deal defending group 1 8325 damage
Immune System group 2 would deal defending group 2 8325 damage

Immune System group 2 attacks defending group 1, killing 1 units
Infection group 1 attacks defending group 2, killing 142 units



In [15]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 191 units
Infection:
Group 1 contains 783 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181656 damage
Immune System group 2 would deal defending group 1 4775 damage
Immune System group 2 would deal defending group 2 4775 damage

Immune System group 2 attacks defending group 1, killing 1 units
Infection group 1 attacks defending group 2, killing 142 units



In [16]:
fight_round(immune_system, infection, debug=True)

Immune system:
Group 2 contains 49 units
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181424 damage
Immune System group 2 would deal defending group 1 1225 damage
Immune System group 2 would deal defending group 2 1225 damage

Immune System group 2 attacks defending group 1, killing 0 units
Infection group 1 attacks defending group 2, killing 49 units



In [17]:
# At this point, all immune system groups are killed.
print(immune_system)
# Only infection groups remain:
print(infection)

[]
[Infection Group 1<Units=782 HP=4706 Attack=116/bludgeoning Init=1 Weak=radiation Immune=NA>, Infection Group 2<Units=4434 HP=2961 Attack=12/slashing Init=4 Weak=cold,fire Immune=radiation>]


In [18]:
sum(g.num_units for g in infection)

5216

In [19]:
import math

def fight(immune_system, infection, max_rounds=math.inf, debug=False):
    ''' A wrapper function that runs right_round() until one army is defeated. '''
    for n in itertools.count():
        fight_round(immune_system, infection, debug)
        if len(immune_system) == 0:
            print('Infection army won in {} rounds'.format(n))
            winning_army = infection
            break
        elif len(infection) == 0:
            print('Immune system won in {} rounds'.format(n))
            winning_army = immune_system
            break
        elif n >= max_rounds:
            print('Breaking after {} rounds'.format(max_rounds))
            return None
    return sum(g.num_units for g in winning_army)

In [20]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
fight(immune_system, infection)

Infection army won in 7 rounds


5216

In [21]:
# The code above works for sample problem. Now try with input data.
with open('input.txt') as input_:
    text = input_.read()
immune_system, infection = parse_text(text)

In [22]:
%time fight(immune_system, infection)

Infection army won in 1370 rounds
CPU times: user 163 ms, sys: 1.77 ms, total: 164 ms
Wall time: 163 ms


16090

# Part 2

In [23]:
def boost(immune_system, amount):
    ''' Add a boost to all the units in the immune system army. '''
    for group in immune_system:
        group.attack_damage += amount

In [24]:
text = '''Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
'''
immune_system, infection = parse_text(text)
boost(immune_system, 1570)
print('\n'.join(map(str,immune_system)))

Immune System Group 1<Units=17 HP=5390 Attack=6077/fire Init=2 Weak=bludgeoning,radiation Immune=NA>
Immune System Group 2<Units=989 HP=1274 Attack=1595/slashing Init=3 Weak=slashing,bludgeoning Immune=fire>


In [25]:
fight(immune_system, infection, debug=True)

Immune system:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Immune System group 2 would deal defending group 1 1577455 damage
Immune System group 2 would deal defending group 2 1577455 damage
Immune System group 1 would deal defending group 2 206618 damage
Infection group 1 would deal defending group 2 185832 damage
Infection group 1 would deal defending group 1 185832 damage
Infection group 2 would deal defending group 1 53820 damage

Infection group 2 attacks defending group 1, killing 9 units
Immune System group 2 attacks defending group 1, killing 335 units
Immune System group 1 attacks defending group 2, killing 32 units
Infection group 1 attacks defending group 2, killing 84 units

Immune system:
Group 1 contains 8 units
Group 2 contains 905 units
Infection:
Group 1 contains 466 units
Group 2 contains 4453 units

Immune System group 2 would deal defending group 1 1443475 damage
Immune System group 2 would 

51

In [26]:
def check_boost(text, amount, max_rounds=math.inf):
    ''' Run the fight with the given boost. '''
    immune_system, infection = parse_text(text)
    boost(immune_system, amount)
    return fight(immune_system, infection, max_rounds)

In [27]:
with open('input.txt') as input_:
    text = input_.read()

In [28]:
check_boost(text, 0)

Infection army won in 1370 rounds


16090

In [29]:
check_boost(text, 1000)

Immune system won in 31 rounds


29516

Okay, so 1000 is big enough to get the immune system to win. Now I will do a manual binary search to find the lowest number. (Searching manually is probably faster than trying to code and test a perfect binary search algorithm.)

In [30]:
check_boost(text, 500)

Immune system won in 61 rounds


28079

In [31]:
check_boost(text, 250)

Immune system won in 116 rounds


25820

In [32]:
check_boost(text, 125)

Immune system won in 217 rounds


21922

In [33]:
check_boost(text, 62)

Immune system won in 445 rounds


14091

In [34]:
check_boost(text, 31)

Immune system won in 928 rounds


9524

In [35]:
# At this point, my binary search plan hits a snag: I find out that when the
# units are evenly matched, the war can result in a deadlock: neither army can
# inflict any damage because the remain units are all immune to the attackers.
check_boost(text, 26)

WarIsDeadlocked: No targets can be found

In [36]:
%%time
# Instead of binary search, I'll just run a sequence of values and look for 
# the best solution.
for amount in range(0,50):
    try:
        outcome = check_boost(text, amount, max_rounds=1_000_000)
    except WarIsDeadlocked as exc:
        print('Boost', amount, 'War is deadlocked: {}'.format(exc))
        continue
    print('Boost', amount, 'Units remaining', outcome)

Infection army won in 1370 rounds
Boost 0 Units remaining 16090
Infection army won in 1369 rounds
Boost 1 Units remaining 15740
Infection army won in 1598 rounds
Boost 2 Units remaining 14993
Infection army won in 1602 rounds
Boost 3 Units remaining 14631
Infection army won in 1602 rounds
Boost 4 Units remaining 14341
Infection army won in 1604 rounds
Boost 5 Units remaining 13857
Infection army won in 1612 rounds
Boost 6 Units remaining 13346
Infection army won in 1613 rounds
Boost 7 Units remaining 12933
Infection army won in 1608 rounds
Boost 8 Units remaining 12524
Infection army won in 1609 rounds
Boost 9 Units remaining 12121
Infection army won in 1612 rounds
Boost 10 Units remaining 11945
Infection army won in 1768 rounds
Boost 11 Units remaining 11476
Infection army won in 1819 rounds
Boost 12 Units remaining 10838
Infection army won in 2232 rounds
Boost 13 Units remaining 9893
Infection army won in 2901 rounds
Boost 14 Units remaining 8414
Boost 15 War is deadlocked: No target

Boost 27 is the lowest value where the immune system wins, so the answer is 8291!